# Análise dados Titanic

Neste Lab vamos analisar os dados do Titanic. Estes dados representam os passageiros do navio e suas características. Podemos dizer que a variável target seria se um passageiro sobreviveu ou não, representado pela variável **Survived**

**Quais hipóteses queremos testar?**

## Vamos começar importando as bibliotecas necessária

In [ ]:
import pandas as pd
import numpy as np

## Overview dos Dados

**Subir CSV para Colab, ler CSV para datataframe e apresentar uma visão geral dos dados**

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.info()

**Visão detalhada das colunas**

In [ ]:
df['Sex'].value_counts()

In [ ]:
df['Age'].value_counts(dropna = False)

In [ ]:
df['Pclass'].value_counts(dropna = False)

In [ ]:
df['Fare'].quantile([0.5])

## Corrigir Inconsistências

Vamos analisar se existem inconsistências nos dados como dados duplicados, valores faltantes entre outros. 

Para cada problema, é necessário analisar se existem padrões e qual a melhor maneira de corrigir.

### Analisar e corrigir dados duplicados

In [ ]:
df.duplicated(subset = ['PassengerId']).sum()

In [ ]:
df = df.drop_duplicates(subset = ['PassengerId'])

In [ ]:
df.duplicated().sum()

### Analisar e corrigir dados faltantes/incorretos

In [ ]:
df['Sex'].unique()

In [ ]:
df['Sex'] =  df['Sex'].str.upper()

In [ ]:
df['Sex'].value_counts(dropna = False)

In [ ]:
df[df['Age'].isna()].head(10)

In [ ]:
len(df[df['Age'].isna()]) / len(df)

In [ ]:
df_age_missing = df[df['Age'].isna()]

In [ ]:
df_age_missing['Sex'].value_counts() / df['Sex'].value_counts()

In [ ]:
df_age_missing['Survived'].value_counts() / df['Survived'].value_counts()

In [ ]:
df.groupby('Sex')['Age'].median()

In [ ]:
df['Parch'].value_counts(dropna = False)

In [ ]:
df.head()

In [ ]:
df['Familia'] = df['Parch'] + df['SibSp']

In [ ]:
(df['Familia'] > 0).value_counts()

In [ ]:
df['TEM_PARENTES'] = df['Familia'] > 0

In [ ]:
df['TEM_PARENTES'].value_counts()

In [ ]:
df.groupby('TEM_PARENTES')['Age'].mean()

In [ ]:
df.groupby('TEM_PARENTES')['Age'].transform('mean')

In [ ]:
df['Age'].fillna(df.groupby('TEM_PARENTES')['Age'].transform('mean'), inplace = True)

In [ ]:
df['Age'].isna().sum()

In [ ]:
df['Embarked'].value_counts()

In [ ]:
portos_titanic = ['S', 'PORT S', 'Q']

In [ ]:
portos_titanic[0]

In [ ]:
conversao = { 'S' : 'PORT S',
             'PORT S' : 'PORT S',
             'C' : 'PORT C',
             'PORT C' : 'PORT C',
             'PORT Q' : 'PORT Q',
             'Q' : 'PORT Q'
}

df['Embarked'] = df['Embarked'].map(conversao)

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df['Pclass'].value_counts(dropna = False)

In [ ]:
df.groupby('Pclass')['Fare'].quantile([0.25, 0.75])

In [ ]:
def preencher_pclass(row):
  if pd.isna(row['Pclass']):
    if row['Fare'] < 15:
      return 3.0
    elif row['Fare'] < 26:
      return 2.0
    else:
      return 1.0
  else:
    return row['Pclass']

In [ ]:
df[df['Pclass'].isna()].iloc[0]

In [ ]:
preencher_pclass(df[df['Pclass'].isna()].iloc[0])

In [ ]:
df['Pclass'] = df.apply(preencher_pclass, axis = 1)

In [ ]:
df['Pclass'].value_counts(dropna = False)

In [ ]:
df[df['Fare'].isna()]['Age'].describe()

In [ ]:
df['Fare'] = np.where(df['Fare'] <= 0, np.nan, df['Fare'])

In [ ]:
df['Fare'].describe()

In [ ]:
df['Fare'].fillna(df.groupby('Pclass')['Fare'].transform('mean'), inplace = True)

In [ ]:
df['Fare'] = np.where(df['Age'] <= 1, 0, df['Fare'])

In [ ]:
df['Fare'].describe()

In [ ]:
df[df['Fare'] == 0]

## Analisar probabilidade de sobrevivência por grupos

In [ ]:
df['Survived'].value_counts()

In [ ]:
df['Survived'].mean()

In [ ]:
df.groupby("Pclass")['Survived'].mean()

In [ ]:
df['Crianca'] = df['Age'] <= 18
df.groupby('Crianca')['Survived'].mean()

In [ ]:
df.groupby(['Crianca', 'Pclass'])['Survived'].mean()

In [ ]:
df.groupby(['Crianca', 'Pclass'])['Survived'].count()